In [2]:
# set up
!pip install statsmodels
!pip install lxml
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.tsa.stattools import coint

In [3]:
# data download from yahoo finance
# Fetch the list of S&P 500 tickers from Wikipedia
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"  
            "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"}
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
tickers_table = pd.read_html(url, storage_options=headers)

"""
print(f"Number of tables found: {len(tickers_table)}")
print("\nExamining all tables:")
for i, table in enumerate(tickers_table):
    print(f"\n--- Table {i} ---")
    print(f"Shape: {table.shape}")
    print(f"Columns: {table.columns.tolist()}")
    print("First few rows:")
    print(table.head(2))
    print("-" * 50)
"""

tickers_df = tickers_table[0]  # This is likely the actual S&P 500 table
print(tickers_df.head())

  Symbol             Security             GICS Sector  \
0    MMM                   3M             Industrials   
1    AOS          A. O. Smith             Industrials   
2    ABT  Abbott Laboratories             Health Care   
3   ABBV               AbbVie             Health Care   
4    ACN            Accenture  Information Technology   

                GICS Sub-Industry    Headquarters Location  Date added  \
0        Industrial Conglomerates    Saint Paul, Minnesota  1957-03-04   
1               Building Products     Milwaukee, Wisconsin  2017-07-26   
2           Health Care Equipment  North Chicago, Illinois  1957-03-04   
3                   Biotechnology  North Chicago, Illinois  2012-12-31   
4  IT Consulting & Other Services          Dublin, Ireland  2011-07-06   

       CIK      Founded  
0    66740         1902  
1    91142         1916  
2     1800         1888  
3  1551152  2013 (1888)  
4  1467373         1989  


In [4]:
sp500_tickers = tickers_df['Symbol'].tolist()
start_date = '2020-01-01'
end_date = '2025-10-01'
# print(sp500_tickers)
df = yf.download(sp500_tickers, start_date, end_date, auto_adjust=True)['Close']

df.head()

[*********************100%***********************]  503 of 503 completed

4 Failed downloads:
['BF.B']: YFPricesMissingError('possibly delisted; no price data found  (1d 2020-01-01 -> 2025-10-01)')
['SOLS', 'Q']: YFPricesMissingError('possibly delisted; no price data found  (1d 2020-01-01 -> 2025-10-01) (Yahoo error = "Data doesn\'t exist for startDate = 1577854800, endDate = 1759291200")')
['BRK.B']: YFTzMissingError('possibly delisted; no timezone found')


Ticker,A,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,ADM,...,WY,WYNN,XEL,XOM,XYL,XYZ,YUM,ZBH,ZBRA,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2020-01-02,82.545311,72.468254,69.823463,NaN,78.208038,41.268997,192.515030,334.429993,108.129158,38.799717,...,23.597670,138.416061,52.264378,53.661701,74.352028,63.830002,91.322357,138.316681,259.140015,127.514328
2020-01-03,81.219986,71.763718,69.160698,NaN,77.254608,41.221451,192.194412,331.809998,106.225708,38.723995,...,23.637821,136.362961,52.515732,53.230316,74.789673,63.000000,91.036331,137.955109,256.049988,127.533371
2020-01-06,81.460060,72.335556,69.706505,NaN,77.659370,41.383106,190.939346,333.709991,104.977654,38.421062,...,23.589638,136.093109,52.440327,53.639004,74.305473,62.570000,90.982704,137.157883,258.010010,126.554260
2020-01-07,81.709770,71.995361,69.308853,NaN,77.227638,41.040779,186.817001,333.390015,107.366005,37.958267,...,23.437090,136.738892,52.331406,53.200027,74.026131,64.589996,91.143585,137.037338,256.470001,126.981972
2020-01-08,82.516502,73.153503,69.800041,NaN,77.542442,40.631893,187.183411,337.869995,108.335693,37.537521,...,23.525408,137.596771,52.281139,52.397758,74.277519,67.599998,91.304443,138.678253,247.639999,126.706345


In [ ]:
sector_info = {}
# Use a sample of tickers to avoid hitting API limits quickly
# In a real-world scenario, you might want to process a large list in batches
for ticker in sp500_tickers:
    try:
        # Fetch the Ticker object
        stock = yf.Ticker(ticker)
        # Get the sector from the stock info
        sector = stock.info.get('sector', 'N/A')
        sector_info[ticker] = sector
    except Exception as e:
        print(f"Could not retrieve info for {ticker}: {e}")
        sector_info[ticker] = 'N/A'


# Convert the sector dictionary to a DataFrame
df_sector = pd.DataFrame(sector_info.items(), columns=['Ticker', 'Sector'])
df_sector = df_sector.set_index('Ticker')

# Combine Price Data with Sector Information, Perform a left join to add the Sector column to the price data
# Transpose the data to have Tickers as rows and Date as columns for easier selection
df = df.T
df_combined = df.merge(df_sector, left_index=True, right_index=True, how='left')

df_combined.head()


In [10]:
# Filter for a Target Sector 
#Tech as example
df_filtered = df_combined[df_combined['Sector'] == 'Technology']

# Drop the 'Sector' column from the final result to show only tickers and prices
df_final = df_filtered.drop(columns=['Sector']).T
df_final.head()

Ticker,AAPL,ACN,ADBE,ADI,ADP,ADSK,AKAM,AMAT,AMD,ANET,...,TER,TRMB,TXN,TYL,UBER,VRSN,WDAY,WDC,XYZ,ZBRA
2020-01-02 00:00:00,72.468254,192.515030,334.429993,108.129158,150.852631,187.830002,87.639999,58.772671,49.099998,12.795000,...,67.934654,41.939999,109.196037,306.239990,30.990000,195.028473,167.460007,48.770992,63.830002,259.140015
2020-01-03 00:00:00,71.763718,192.194412,331.809998,106.225708,150.533813,184.949997,87.239998,57.837223,48.599998,12.553125,...,66.520988,41.509998,107.746483,306.670013,31.370001,199.142578,168.440002,48.020329,63.000000,256.049988
2020-01-06 00:00:00,72.335556,190.939346,333.709991,104.977654,150.737534,187.119995,87.550003,56.589958,48.389999,12.678750,...,65.078049,41.310001,106.996429,310.209991,31.580000,200.986496,169.490005,47.098717,62.570000,258.010010
2020-01-07 00:00:00,71.995361,186.817001,333.390015,107.366005,148.913010,187.500000,90.199997,58.224628,48.250000,12.803125,...,66.511230,41.150002,109.061195,311.269989,32.810001,201.452438,172.949997,50.287189,64.589996,256.470001
2020-01-08 00:00:00,73.153503,187.183411,337.869995,108.335693,150.312378,189.949997,91.400002,58.186832,47.830002,12.936250,...,66.589211,41.320000,109.356133,310.989990,33.930000,202.384293,178.710007,51.015560,67.599998,247.639999


In [11]:
# data cleaning (Remove stocks with too many missing values (>20%); Forward fill remaining missing values (or interpolate)

df_clean = df_final.dropna(axis=1)
df_clean = df_clean.fillna(method='ffill')


C:\Users\Administrator\AppData\Local\Temp\ipykernel_4168\1859513710.py:4: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_clean = df_clean.fillna(method='ffill')


In [12]:
# Cointegration Test for Pair Selection - Engle-Granger cointegration test
log_prices = np.log(df_clean)

def find_cointegrated_pairs(data):
    n = data.shape[1]
    tickers = data.columns
    coint_pairs = []

    for i in range(n):
        for j in range(i+1, n):
            pvalue = coint(data.iloc[:, i], data.iloc[:, j])[1]
            if pvalue < 0.05:  # statistically significant at 5%
                coint_pairs.append((tickers[i], tickers[j], pvalue))
    return sorted(coint_pairs, key=lambda x: x[2])

coint_pairs = find_cointegrated_pairs(log_prices)
print("Number of cointegrated pairs found:", len(coint_pairs))

# show top 10 pairs
coint_pairs[:10]
    

Number of cointegrated pairs found: 194


[('AMAT', 'NXPI', np.float64(1.7735311957506104e-05)),
 ('FTNT', 'PAYX', np.float64(0.0012602955965562773)),
 ('INTU', 'NTAP', np.float64(0.0012627054132550379)),
 ('ADI', 'HPE', np.float64(0.0015322253053625748)),
 ('AAPL', 'HPE', np.float64(0.0015844563276472814)),
 ('ADI', 'MPWR', np.float64(0.0019200396756336798)),
 ('CDNS', 'JBL', np.float64(0.0021119143370804524)),
 ('CSCO', 'TDY', np.float64(0.002127661731173009)),
 ('DELL', 'MSFT', np.float64(0.0021495269774370633)),
 ('CDNS', 'PANW', np.float64(0.0022596638321574418))]